### HWNI Boot Camp - TMS introduction and demo 

The following jupyter notebook is a quick demonstration of how a TMS study is designed, implemented, and analyzed. 

#### You can 'run' each cell by pressing _shift_ and *return* at the same time 

In [ ]:
# import python stuff 
import numpy as np
np.set_printoptions(precision = 3)
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
pd.set_option('precision',2)
import scipy.io as io
import scipy.stats as stats
import seaborn as sns
sns.set(color_codes=True)
sns.set_style('white')
import glob
import nilearn
nilearn.__version__
from IPython.display import Image

Every study starts with a research question. You'll see ours in the cell below. 

The experimental task we gave participants was to remember a direction (*left / right / up / down*). While they remembered this direction, they had to perform a movement based upon a color. This movement could be in the same (comaptible) or different (incompatible) direction as the word they are remembering. 

In general, people are much less accurate and respond more slowly when movements are _incompatible_. We've called this the compatibility effect. 

In [ ]:
Image('figures/task.png')

#### We had participants perform 9 blocks of this task, for an hour-long experiment. 

#### In each of the blocks, the ratio of compatible to incompatible trials was different. The more compatible trials, the more likely, we think, people are to make mistakes, because the memory item is genrally useful. 

#### This is akin to a *Stroop* effect, but here the interference is between memory and action, as opposed to percpetion and language. 

In [ ]:
Image('figures/gating.png')

### We hypothesized the the *lateral prefrontal cortex* would be involved in controlling the interaction between working memory and motor behavior. 

#### To determine if this is the case, we had participants perform the task after TMS to the lateral PFC. Participants did the experiment 3 times: 
- continuous theta-burst TMS ==> reduce PFC activity
- intermittent theta-burst TMS ==> boost PFC activity 
- no TMS ==> 'normal' PFC activity 

#### See an outline of our methods below

In [ ]:
Image('figures/study_design.png')

#### Why do you think different patterns of TMS produce different effects on neural excitability? What do you think these frequency-specific patterns are based on?

*See Huang et al (2005) if you want to find out more about why:* https://www.sciencedirect.com/science/article/pii/S0896627304008463

Let's plot 20 s of both cTBS and iTBS below, for a closer look at these types of TMS

In [ ]:
fig, ax = plt.subplots(2, figsize = (15,6))
plt.subplots_adjust(hspace=0.5)

cTBS_length = 40
n_bursts = 200
n_pulses = 600 

ax[0].set_xlim(0,cTBS_length/2)
for i in np.arange(0,cTBS_length,1/5):
    ax[0].axvline(x=i, ymin=0.2, ymax = 0.8, color = 'orange')
ax[0].set_title('20 seconds of continuous theta-burst', fontsize=16)
ax[0].set_xlabel('time (s)', fontsize=12)
    
ax[1].set_xlim(0,cTBS_length/20*5*2)
for i in np.arange(0,2,1/5):
    ax[1].axvline(x=i, ymin=0.2, ymax = 0.8, color = 'lightgreen')
    ax[1].axvline(x=i+1/50, ymin=0.2, ymax = 0.8, color = 'lightgreen')
    ax[1].axvline(x=i+2/50, ymin=0.2, ymax = 0.8, color = 'lightgreen')
    
for i in np.arange(10,12,1/5):
    ax[1].axvline(x=i, ymin=0.2, ymax = 0.8, color = 'lightgreen')
    ax[1].axvline(x=i+1/50, ymin=0.2, ymax = 0.8, color = 'lightgreen')
    ax[1].axvline(x=i+2/50, ymin=0.2, ymax = 0.8, color = 'lightgreen')
ax[1].set_title('20 seconds of intermittent theta-burst', fontsize=16)
ax[1].set_xlabel('time (s)', fontsize=12)

fig, ax = plt.subplots(1, figsize = (3,3))
ax.set_xlim(0,.1)
for i in np.arange(0,0.061,0.02):
    ax.axvline(x=i, ymin=0.2, ymax = 0.8, color = 'blue')
ax.set_title('Each "burst" of TMS is 3 pulses in rapid succession (50 Hz)', fontsize=16)
ax.set_xlabel('time (s)', fontsize=12)

#### Now, let's show where the TMS stimulation took place in an example TMS subject. After taking the MRI, we locate the prefrontal cortex by comapring this individual's brain anatomy to a template. 

#### See the example subject's anatomical (T1w) MRI below. 

In [ ]:
from nilearn import plotting, image
#fig,ax = plt.subplots(1,figsize=(12,3))
plotting.plot_anat('figures/sub-127_T1w.nii.gz', display_mode = 'z', cut_coords = np.arange(-15,75,15))

#### Now let's add the specific point of the lateral PFC we're interested in targeting. As I showed, earlier, this is the target which we aim for during the stimulation itself

In [ ]:
fig,ax = plt.subplots(1,figsize=(10,5))
display = plotting.plot_anat('figures/sub-127_T1w.nii.gz', display_mode = 'ortho',
                             cut_coords=(-39, 54, 42), figure=fig)
display.add_markers([(-39, 54, 42)], marker_color='orange', marker_size=75) 

### Looking at the results: let's load the data from each subject and determine the effect of TMS on the behavioral experiment

In [ ]:
subjects = ['OGWM_' + str(i) for i in range(101,127)] # OGWM_101 - OGWM_120
df_combined = pd.DataFrame() # empty DataFrame 
df_means = pd.DataFrame()
df_all_trials = pd.DataFrame()


In [ ]:
# loop through and groupby compatibility and block to make new dataframe, 
#  from which a factor plot with variance can be plotted

# remove subjects if excluded
subjects.remove('OGWM_102')
subjects.remove('OGWM_120')
subjects.remove('OGWM_121')

TMS_sessions = ['no_TMS', 'PFC_cTBS', 'PFC_iTBS']


for sub in subjects:
    
    for TMS_session in TMS_sessions:
        
        # get filename for given TMS session
        fname = 'OGWM_word_SD_block_comp_TBS_{}_*_{}.txt'.format(sub,TMS_session)
        glob.glob(fname)
        fname = glob.glob(fname)[0]
        session = fname.split('_')[8]+'_'+fname.split('_')[9] # sess_1 / sess_2 / sess_3
        fname = 'OGWM_word_SD_block_comp_TBS_{}_{}_{}.txt'.format(sub,session,TMS_session)
    
        #fname = 'OGWM_word_SD_block_comp_{}.txt'.format(sub)
        df = pd.read_csv(fname, header = 0, sep = '\s+') # read in df for the subject
        # df = df[(df.ACC == 1) & (df.probeACC == 1)] # keep only correct trials 
        df = df[['subject','session','target','block','BlockType','TrialType','move_init_bound_msecRT', 'enter_box_msecRT', 'velocity_drop_msecRT', 'msecRT', 'ACC', 'probemsecRT', 'probeACC']]
        df['msecRT'] = pd.to_numeric(df['msecRT'], errors = 'coerce')
        df['move_init_bound_msecRT'] = pd.to_numeric(df['move_init_bound_msecRT'], errors = 'coerce')
        df['enter_box_msecRT'] = pd.to_numeric(df['enter_box_msecRT'], errors = 'coerce')
        df['velocity_drop_msecRT'] = pd.to_numeric(df['velocity_drop_msecRT'], errors = 'coerce')
        df['probemsecRT'] = pd.to_numeric(df['probemsecRT'], errors = 'coerce')

        # add column for corrected movements from text file for each sub 
        corrections_fname = 'OGWM_word_SD_block_comp_TBS_{}_{}_{}_corrected_movements.txt'.format(sub,session,TMS_session)    
        corrections_fname_1_4 = 'OGWM_word_SD_block_comp_TBS_{}_{}_{}_corrected_movements_1_4.txt'.format(sub,session,TMS_session)
        angles_fname = 'OGWM_word_SD_block_comp_TBS_{}_{}_{}_movement_angle.txt'.format(sub,session,TMS_session)    
        angles_fname_1_4 = 'OGWM_word_SD_block_comp_TBS_{}_{}_{}_movement_angle_1_4.txt'.format(sub,session,TMS_session)    
        df['corrected_movements'] = np.genfromtxt(corrections_fname)
        df['corrected_movements_1_4'] = np.genfromtxt(corrections_fname_1_4)
        df['movement_angle'] = np.genfromtxt(angles_fname)
        df['movement_angle_1_4'] = np.genfromtxt(angles_fname_1_4)
        df['movement_angle_difference'] = 180-np.abs(df['movement_angle'])
        df['movement_angle_difference_1_4'] = 180-np.abs(df['movement_angle_1_4'])
        df_combined = pd.concat([df_combined, df])

### Report subject accuracies for motor and probe responses

In [ ]:
df_orig = df_combined

Below is the order of the block types for each participant

In [ ]:
for sub in subjects:
    print(sub, list(df_combined.groupby(('subject')).get_group(sub).drop_duplicates(subset='block',keep='first').BlockType))

In [ ]:
grouped_subject = df_combined.groupby('subject') # group by subject name 

#### Are participants making more errors on compatible trials? Now let's comapre the accuracy on compatible versus incomaptible trials

In [ ]:
# plot accuracy by trial type 

size=8
fs=16

group_acc_plot = df_combined.groupby(['subject', 'target','BlockType', 'TrialType'], as_index=False).aggregate(np.mean).groupby('BlockType').get_group('neutral')

#g = sns.factorplot(x='TrialType', y = 'ACC', data = group_acc_plot, palette = 'Set2', kind = 'box', legend = False, size = size, aspect = 0.7, width = 0.4)
#sns.stripplot(x='TrialType', y = 'ACC', data = group_acc_plot, alpha = 0.7, size = 9, color = 'gray', edgecolor = 'k', linewidth = 2)
g = sns.factorplot(x='TrialType', y = 'ACC', col='target',data = group_acc_plot, palette = 'Set2', kind = 'point', legend = False, 
                   aspect = 0.7, size = size, scale = 2.5, errwidth = 4)

# labels 
g.set_xticklabels(['compatible','incompatible'], fontsize = fs + 2)
g.set_xlabels('Trial Type', fontsize = fs + 2)
g.set_ylabels('Accuracy', fontsize = fs+2)
g.set_yticklabels(fontsize = fs - 1)
# setting ticks 
#plt.grid(b=True, which='major', color='k', linewidth=1, axis = 'y', alpha = 0.3)
plt.tight_layout()
#plt.ylim(0.931,1.0)
#plt.yticks(np.arange(0.93,1,0.1))
#plt.minorticks_on()
#plt.grid(b=True, which='minor', color='k', linewidth=0.5, axis = 'y', alpha = 0.3, linestyle = '--')
sns.despine(right = False, top = False)
#plt.savefig('OGWM_block_boxplot_{}.png'.format(suffix_2), dpi=300)

# test difference in accuracy by trial type (across blocks) 
comp_means = group_acc_plot[group_acc_plot.TrialType=='compatible'].probeACC
incomp_means = group_acc_plot[group_acc_plot.TrialType=='incompatible'].probeACC

# # t-test of subbject means for comp vs incomp conditions
# t, p = stats.ttest_rel(comp_means,incomp_means)
# print('T-test of ACC (compatible vs. incompatible subject means): \n t = {0} \n p = {1}'.format(t,p))

#### Above, trial types are compared across TMS sessions. Now, let's test all the variables in out experiment:
- TMS: *cTBS vs. iTBS vs. no TMS*
- trial type: *compatible vs. incompatible*
- block type: *valid vs. neutral vs. invalid*

In [ ]:
group_acc_plot = df_combined.groupby(['subject', 'target','BlockType', 'TrialType'], as_index=False).aggregate(np.mean)
group_acc_plot

In [ ]:
sns.factorplot(x='TrialType', y = 'ACC', col='BlockType', row = 'target', data = group_acc_plot, 
                   palette = 'Set2', kind = 'point', legend = False, 
                   aspect = 0.75, errwidth = 4, size = 5, scale = 2, ci = 68)

#### We can do a basic quantitative analysis to combine these factors in an ANOVA, a form of linear model

In [ ]:
import statsmodels 
from statsmodels.stats.anova import AnovaRM

acc_anova = df_combined.groupby(['subject', 'target','BlockType', 'TrialType'], as_index=False).aggregate(np.mean)
anvRM = AnovaRM(acc_anova, 'ACC', 'subject', within=['target','TrialType', 'BlockType'])
fit = anvRM.fit()
#print(DV_measure, ': ', suffix_2, suffix)
fit.summary()

# within each trial tyoe, run an interaction!!

#### What effects seem most prominent in the data, and why do you think that might be?

### If there is extra time, run all of the cells below to see a graph of reaction time measures. What do you think is the best way to measure reaction time in this experiment? And how might the results be different from accuracy?

### Report RT measures - begin dropping incorrect trials for motor movement and probe 

In [ ]:
error_trials = True

if error_trials == False:
    df_combined = df_combined[(df_combined.ACC == 1) & (df_combined.probeACC == 1)] 
    suffix_2 = 'w_o_errors'
else: 
    df_combined = df_combined[(df_combined.ACC == 1) & (df_combined.probemsecRT != 9999000)] 
    suffix_2 = 'w_errors'

# include only accurate trials, filter out any trials with ACC or probeACC not equal to 1
#df_combined = df_combined[(df_combined.ACC == 1) & (df_combined.probeACC == 1)] 
# include inaccurate trials but exclude nonresponse trials 
#df_combined = df_combined[(df_combined.ACC == 1) & (df_combined.probemsecRT != 9999000)] 

#### Trim subject RT distributions for +/- 3 SD

In [ ]:
def trim_outlier_RTs(df_data, DV):
    """
    Loop through subjects incombined dataframe and trim outlierrs +/- 3 SD from mean RT within each subject
    
    df_data: Pandas DataFrame concatenated across subjects 
    DV: dependent variable to trim 
    """
    
    df_combined_trimmed = pd.DataFrame()
    subjects = df_data.subject.unique()
    
    for sub in subjects:
        df_sub = df_data[df_data.subject == sub]
        df_sub_trimmed = trim_outliers_subject(df_sub, DV)
        df_combined_trimmed = pd.concat([df_combined_trimmed, df_sub_trimmed])
        
    print('Original data shape: ', df_data.shape)
    print('Trimmed data shape: ', df_combined_trimmed.shape)
    return df_combined_trimmed 

def trim_outliers_subject(df_sub, DV):
    """
    Within a given subject's data, trim RTs +/- 3 SD from mean RT
    """
    rt_dist = df_sub[DV]
    rt_mean = np.mean(rt_dist)
    rt_std = np.std(rt_dist)
    
    bool_outliers = np.abs(rt_dist - rt_mean) > rt_std*3 # boolean indices for the outliers 
    df_sub_trimmed = df_sub[np.logical_not(bool_outliers)] # keep all data that is NOT outlier 
    
    outliers = rt_dist[np.abs(rt_dist - rt_mean) > rt_std*3]
    print('Outlier {} for subject {}: \n {}\n'.format(DV, df_sub.subject.unique(),outliers))
    #rt_include = rt_dist[np.logical_not(np.absolute(rt_dist - rt_mean) > rt_std*3)]
    
    return df_sub_trimmed 

In [ ]:
# Add a movement time parameter
df_combined['movement_time_msecRT'] = df_combined['msecRT'] - df_combined['move_init_bound_msecRT']

In [ ]:
df_combined_trimmed = trim_outlier_RTs(df_combined, 'move_init_bound_msecRT')
df_combined_trimmed = trim_outlier_RTs(df_combined_trimmed, 'movement_time_msecRT')

In [ ]:
# calculate proportion of outlier trials removed 
(df_combined.shape[0] - df_combined_trimmed.shape[0])/df_combined.shape[0]

In [ ]:
grouped_subj_TT = df_combined_trimmed.groupby(['subject', 'TrialType'], as_index=False)
grouped_subj_TT_means = grouped_subj_TT.aggregate(np.mean)

### Report RT measures by block types 
### Set DV measure - total RT, movement initiation, movement time, corrected movements

In [ ]:
DV = 'move_init_bound_msecRT'
#DV = 'movement_time_msecRT'
#DV = 'msecRT'
#DV = 'correction_proportion'
#DV = 'correction_proportion_1_4'
#DV = 'movement_angle_difference'
#DV = 'movement_angle_difference_1_4'

corrected_movements = True  # True/False to include corrected movements 
corrected_movements_1_4 = True

pal = [(0.40000000000000002, 0.76078431372549016, 0.6470588235294118),
 (0.55294117647058827, 0.62745098039215685, 0.79607843137254897),
           (0.9882352941176471, 0.55294117647058827, 0.3843137254901961)]

In [ ]:
print(df_combined_trimmed.shape)

if corrected_movements == False:
    if corrected_movements_1_4 == False:
        df_analyses = df_combined_trimmed[df_combined_trimmed.corrected_movements==0]
    else:
        df_analyses = df_combined_trimmed[df_combined_trimmed.corrected_movements_1_4==0]
    suffix = 'w_o_corrections'
elif corrected_movements == True and corrected_movements_1_4 == False:
    df_analyses = df_combined_trimmed[df_combined_trimmed.corrected_movements_1_4==0]
    suffix = 'w_o_corrections_1_4'
else: 
    df_analyses = df_combined_trimmed
    suffix = 'w_corrections'

    
if corrected_movements_1_4 == True:
    suffix += '_1_4'
    
print(df_analyses.shape)
    
# if corrected_movements_1_4 == False:
#     df_analyses = df_combined_trimmed[df_combined_trimmed.corrected_movements==0]
#     suffix = 'w_o_corrections'
# else: 
#     df_analyses = df_combined_trimmed
#     suffix = 'w_corrections'

In [ ]:
grouped_subj_TT_BT = df_analyses.groupby(['subject','target','TrialType','BlockType'], as_index=False)
grouped_subj_TT_BT_means = grouped_subj_TT_BT.aggregate(np.mean)
grouped_subj_TT_BT_sum = grouped_subj_TT_BT.aggregate(np.sum)

In [ ]:
#grouped_subj_TT_BT_means.to_csv('OGWM_TMS_aggregated_mean_no_outliers.csv')

In [ ]:
## counts of corrected movements

block_types = df_combined.BlockType.unique()
trial_types = df_combined.TrialType.unique()
target_types = df_combined.TrialType.unique()

import itertools
block_trial_combinations = list(itertools.product(trial_types, block_types, target_types))

subjects = df_combined.subject.unique()

grouped_subj_TT_BT_sum['trial_count'] = np.nan
#trial_counts = np.empty(shape=(len(subjects),len(block_trial_combinations)))

# for i, sub in enumerate(subjects):
#     #print(sub)
#     df_sub = df_combined[df_combined.subject == sub]
#     df_sub_grouped = df_sub.groupby(['TrialType','BlockType']) 
#     for j, combo in enumerate(block_trial_combinations): 
#         df_combo = df_sub_grouped.get_group(combo)
#         trials = df_combo.shape[0]
        
#     df_sub_grouped = df_sub.groupby(['TrialType','BlockType']) 

for index, row in grouped_subj_TT_BT_sum.iterrows():
    
    sub = row.subject
    BT = row.BlockType
    TT = row.TrialType
    target = row.target
    
    df_sub = df_orig[(df_orig.subject == sub)&(df_orig.BlockType == BT)&(df_orig.TrialType == TT)&(df_orig.target == target)]
    trials = df_sub.shape[0]
    
    mask = (grouped_subj_TT_BT_sum.subject==sub)&(grouped_subj_TT_BT_sum.BlockType == BT)&(grouped_subj_TT_BT_sum.TrialType == TT)&(grouped_subj_TT_BT_sum.target == target)
    grouped_subj_TT_BT_sum.loc[mask, 'trial_count'] = trials

    #grouped_subj_TT_BT_sum.loc[index].trial_count = trials 
        
#grouped_subj_TT_BT_sum

In [ ]:
# divide number of corrections by number of trials in condition
grouped_subj_TT_BT_sum['correction_proportion'] = grouped_subj_TT_BT_sum['corrected_movements']/grouped_subj_TT_BT_sum['trial_count']
grouped_subj_TT_BT_sum['correction_proportion_1_4'] = grouped_subj_TT_BT_sum['corrected_movements_1_4']/grouped_subj_TT_BT_sum['trial_count']
#grouped_subj_TT_BT_sum.to_csv('OGWM_TMS_aggregated_sum_no_outliers.csv')

### box plot based on DV 

In [ ]:
fs = 24
if DV == 'correction_proportion' or DV == 'correction_proportion_1_4': 

    #DV_graph = 'correction_proportion'
    
    data = grouped_subj_TT_BT_sum # use data of SUMS
    
    fig, ax = plt.subplots(1,3, sharey=True, figsize = (15,9)) 

    types = ['valid', 'neutral', 'invalid']
    x_labels = ['80% (valid) block', '50% (neutral) block', '20% (invalid) block']

    for i in range(len(types)):

        # ax1
        #data = data[data.BlockType==types[i]]
        g=sns.factorplot(x='TrialType', y = DV,  row = 'target', data = data.groupby('BlockType').get_group(types[i]), palette = 'Set2', 
                            kind = 'factor',legend = False, size = size, aspect = 0.7, ax = ax[i], ci=95,
                        scale = 1.5,errwidth=4)
        #sns.stripplot(x='TrialType', y = DV_graph,  data = data.get_group(types[i]), alpha = 0.7, size = 7, 
        #              color = 'gray', edgecolor = 'k', linewidth = 2, ax = ax[i])
        # labels 
        #ax[i].plot(0,data.DVs[i],'o')
        ax[i].tick_params(which='both', labelsize=fs-2)
        ax[i].set_xticklabels(['compatible','incompatible'], fontsize = fs + 2)
        ax[i].set_xlabel(x_labels[i], fontsize = fs + 2)
        ax[i].set_ylabel('Proportion of corrected movements', fontsize = fs+1)
        #ax1.set_yticklabels(fontsize = fs - 1)
        # setting ticks 
        ax[i].grid(b=True, which='major', color='k', linewidth=1, axis = 'y', alpha = 0.1)
        ax[i].minorticks_on()
        ax[i].grid(b=True,which='minor', color='k', linewidth=0.5, axis = 'y', alpha = 0.1, linestyle = '--')
        sns.despine(right = False, top = False)
        plt.close(g.fig)
    
else: 
    
    data = grouped_subj_TT_BT_means # use data of MEANS

    for TMS_session in TMS_sessions:
        
        data_session = data[data.target==TMS_session]
        
        fig, ax = plt.subplots(1,3, sharey=True, figsize = (10,6)) 
        plt.ylim(400,600)
        plt.suptitle(TMS_session)

        types = ['valid', 'neutral', 'invalid']
        x_labels = ['80% (valid) block', '50% (neutral) block', '20% (invalid) block']

        for i in range(len(types)):

            g=sns.factorplot(x='TrialType', y = DV, data = data_session[data_session.BlockType==types[i]], palette = 'Set2', 
                            kind = 'point', legend = False, size = size, aspect = 0.7, ax = ax[i], ci=68, 
                            scale = 2.5, errwidth = 4, alpha=1)
            #sns.stripplot(x='TrialType', y = DV,  data = data[data.BlockType==types[i]], alpha = 0.7, size = 7, 
            #              color = 'gray', edgecolor = 'k', linewidth = 2, ax = ax[i])
            # labels 
            #ax[i].plot(0,data.DVs[i],'o')
            ax[i].tick_params(which='both', labelsize=fs-2)
            ax[i].set_xticklabels(['compatible ','incompatible'], fontsize = fs-10)
            ax[i].set_xlabel(x_labels[i], fontsize = fs - 6)
            if i == 0:
                ax[i].set_ylabel(DV, fontsize = fs+1)
            #ax1.set_yticklabels(fontsize = fs - 1)
            # setting ticks 
            ax[i].grid(b=True, which='major', color='k', linewidth=1, axis = 'y', alpha = 0.1)
            #ax[i].minorticks_on()
            #ax[i].grid(b=True,which='minor', color='k', linewidth=0.5, axis = 'y', alpha = 0.1, linestyle = '--')
            sns.despine(right = False, top = False)
            plt.close(g.fig)
            #plt.savefig

            #plt.savefig('OGWM_factorplot_blocktypes_{}_{}_{}.png'.format(DV,suffix,suffix_2), dpi=300)